In [178]:
using Pkg
Pkg.activate("/home/gribeill/GitHub/HiQuER/")
using HiQuER
using LRUCache
using BenchmarkTools
using OpenQASM
using StaticArrays

  Activating project at `~/GitHub/HiQuER`


In [179]:
using Revise

In [180]:
struct Test{Sz} where Sz<:Int
    q::StaticArray{Sz, Float64}
end

LoadError: syntax: invalid type signature around In[180]:1

In [181]:
T == T'

false

In [205]:
qasm = """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];

rz(-0.2132411223) q[0];
rz(0.12324211) q[1];
"""

open("test/fold.qasm", "w") do io
    write(io, qasm)
end

94

In [206]:
macro bash_str(s) open(`bash`,"w",stdout) do io; print(io, s); end; end

@bash_str (macro with 1 method)

In [207]:
bash"""
cat test/fold.qasm | python3 ../util/rotation_synth/rotation_synth.py | \
tee test/fold2.qasm | staq_resource_estimator
"""

Resources used:
  depth: 153
  t: 120
  h: 120
  y: 1
  s: 50
  qubits: 2


In [208]:
bash"""
cat test/fold2.qasm | staq_lattice_surgery > fold2.json"""

In [190]:
qq = OpenQASM.parse(qasm)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
rz(-(0.2132411223)) q[0];
rz(-(0.12324211)) q[1];

In [188]:
(qq.prog[5])

LoadError: MethodError: no method matching fieldnames(::OpenQASM.Types.Measure)
[0mClosest candidates are:
[0m  fieldnames([91m::Core.TypeofBottom[39m) at reflection.jl:188
[0m  fieldnames([91m::Type{<:Tuple}[39m) at reflection.jl:190
[0m  fieldnames([91m::DataType[39m) at reflection.jl:185
[0m  ...

In [173]:
using RBNF

In [175]:
qq.prog[3].cargs[1][2] isa RBNF.Token

true

In [170]:
qq.prog[3].cargs[1]

(-(pi), RBNF.Token{:reserved}(6, 7, 56, "/", 1), RBNF.Token{:int}(6, 8, 57, "2", 1))

In [169]:
typeof(qq.prog[3].cargs[1])

Tuple{OpenQASM.Types.Neg, RBNF.Token{:reserved}, RBNF.Token{:int}}

In [163]:
parse_qasm_circuit(qasm)

u3(0.4721, 0.31211, 0.1) q[1];


1: Rz(0.1)(2)
2: Rx(π/2)(2)
3: Rz(3.4537026535897932)(2)
4: Rx(π/2)(2)
5: Rz(9.896877960769379)(2)

In [133]:
a.value

3152519739159347//4503599627370496

In [127]:
using Match

In [134]:
x = "a"
n = @match x begin
    "a" => begin 
        z = 1
        q = 3
        z+q
    end
    "b" => 3
end

4

In [135]:
n

4

In [123]:
HiQuER.parse_qasm(qq.prog[3])

(nothing, [("q", 2)])

In [301]:
function load_qasm(path)
    open(path, "r") do io
        qasm = read(io, String)
        return OpenQASM.parse(qasm)
    end 
end

function parse_qasm(reg::OpenQASM.Types.RegDecl)
    name = reg.name.str
    width = parse(Int, reg.size.str)
    return (HiQuER.QRegister(name, width),)
end

function parse_qasm(bit::OpenQASM.Types.Bit)
    return (bit.name.str, 1+parse(Int, bit.address.str))
end 

function parse_carg(carg)
    if carg isa OpenQASM.Types.Neg
        return -1.0*parse(Float64, carg.val.str)
    else
        return parse(Float64, carg.str)
    end
end
    

function parse_qasm(instr::OpenQASM.Types.Instruction)
    qargs = [parse_qasm(q) for q in instr.qargs]
    cargs = instr.cargs
    gate = @match instr.name begin
        
        "s" => HiQuER.S
        "sdg" => HiQuER.S'
        "h"   => HiQuER.H
        "t"   => HiQuER.T
        "tdg" => HiQuER.T'
        "cx"  => HiQuER.CNOT
        "rx"  => HiQuER.Rx(FloatAngle(parse_carg(cargs[1])))
        "ry"  => HiQuER.Ry(FloatAngle(parse_carg(cargs[1])))
        "rz"  => HiQuER.Rz(FloatAngle(parse_carg(cargs[1])))
    end
    
    return (gate, qargs)
end

function parse_qasm(include::OpenQASM.Types.Include)
    return nothing
end

function load_qasm_circuit(path)
    ast = load_qasm(path)
    prog = ast.prog
    
    c = Circuit()
    
    for (idx, instr) in enumerate(prog)
        println(idx, instr)
        if !isnothing(local m = parse_qasm(instr))
            push!(c, m...)
        end
    end
    return c
end 

load_qasm_circuit (generic function with 1 method)

In [21]:
varinfo(r"z")

| name |      size | summary |
|:---- | ---------:|:------- |
| z    | 2.048 MiB | Circuit |


In [23]:
[1,2,3]*[3,4,5]

LoadError: MethodError: no method matching *(::Vector{Int64}, ::Vector{Int64})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at ~/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/matmul.jl:49
[0m  *(::StridedVecOrMat, [91m::LinearAlgebra.Adjoint{<:Any, <:LinearAlgebra.LQPackedQ}[39m) at ~/julia-1.8.2/share/julia/stdlib/v1.8/LinearAlgebra/src/lq.jl:269
[0m  ...

In [26]:
a = Set([1,2,3])
b = Set([1,4,5])

Set{Int64} with 3 elements:
  5
  4
  1

In [49]:
gates = Vector{HiQuER.AbstractGate}()

HiQuER.AbstractGate[]

In [40]:
cliffordT = Set([H, S, T, X180, Y180, Z180, X90, Y90, Z90, CNOT])
union!(cliffordT, Set([g' for g in cliffordT]));

In [69]:
ϵ = 1e-12
θ = pi/3
out = readchomp(`gridsynth -e $ϵ -p 3\*pi/41`)

"HTHTSHTSHTHTSHTSHTSHTSHTSHTHTHTHTHTSHTHTHTHTHTHTHTHTHTSHTHTSHTHTSHTSHTHTHTSHTSHTSHTSHTHTHTSHTSHTSHTHTSHTHTSHTSHTSHTHTHTHTSHTSHTHTSHTSHTHTHTHTHTSHTHTSHTHTSHTHTHTHTSHTHTHTSHTHTSHTHTSHTHTHTHTHTSHTHTSHTSHTHTHTSHTHTSHTSHTHTSHTHTSHTHTSHTSHTHTSHTSHTHTHTSHTSHTHTSHTSHTHTSHTHTHTHTHTSHTHTSHTHTHTSHSSS"

In [102]:
function gridsynth(angle::Float64, ϵ=1e-10)
    decomp = readchomp(`gridsynth -e $ϵ -p $angle`)
    gates = Vector{HiQuER.AbstractGate}()
    for u in decomp
        if u == 'X'
            push!(gates, X180)
        else
            push!(gates, eval(Symbol(u)))
        end
    end
    return gates
end

gridsynth (generic function with 4 methods)

In [97]:
const synth_lru = LRU{Tuple{Float64, Float64}, Vector{HiQuER.AbstractGate}}(maxsize=10000)

function cached_gridsynth(θ::FloatAngle, ϵ=1e-10)
    angle = θ.value
    old = filter(x -> abs(x[1] - angle) < ϵ, collect(keys(synth_lru)))
    if length(old) > 0
        angle = old[1][1]
    end
    get!(synth_lru, (angle, ϵ)) do
        gridsynth(angle, ϵ)
    end
end     

cached_gridsynth (generic function with 2 methods)

In [110]:
@benchmark gridsynth(1.0 + pi/32815)

BenchmarkTools.Trial: 377 samples with 1 evaluation.
 Range (min … max):  11.372 ms … 22.197 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.568 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.265 ms ±  1.878 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃▄█▄                                                      
  ▃▆█████▇▇█▆▇▄▄▅▄▄▄▄▃▃▃▄▄▃▃▃▃▁▃▃▂▃▃▂▁▃▃▁▁▂▂▂▂▂▃▃▁▂▂▃▂▂▁▂▁▁▁▂ ▃
  11.4 ms         Histogram: frequency by time        20.4 ms <

 Memory estimate: 225.37 KiB, allocs estimate: 572.

In [100]:
cached_gridsynth(FloatAngle(pi/127 + 1e-9 + 1e-10 + 1e-7))

being slow


248-element Vector{HiQuER.AbstractGate}:
 H
 T
 S
 H
 T
 S
 H
 T
 H
 T
 S
 H
 T
 ⋮
 T
 H
 T
 H
 T
 S
 H
 T
 H
 X
 S
 S

In [53]:
for (idx, slice) in enumerate(c.slices)
    nonCT = filter(x -> !(x in cliffordT), collect(values(slice.gates)))
    append!(gates, nonCT)
end

In [54]:
gates

958-element Vector{HiQuER.AbstractGate}:
 Rz(0.12120496481657)
 Rz(0.112213991582394)
 Rz(0.0930429250001907)
 Rz(0.125459060072899)
 Rz(0.103709444403648)
 Rz(0.128557279706001)
 Rz(0.116796024143696)
 Rz(0.123350352048874)
 Rz(0.109851323068142)
 Rz(0.094442144036293)
 Rz(0.107436716556549)
 Rz(0.1133768633008)
 Rz(0.124409168958664)
 ⋮
 Rz(-0.0476885512471199)
 Rz(-0.0490914508700371)
 Rz(-0.0515644103288651)
 Rz(-0.0486283153295517)
 Rz(-0.0509018823504448)
 Rz(-0.0426804572343826)
 Rz(-0.0479252263903618)
 Rz(-0.0524347499012947)
 Rz(-0.052650086581707)
 Rz(-0.0506791174411774)
 Rz(-0.0513445138931274)
 Rz(-0.0497780814766884)

In [136]:
a = Set([1,2,3])

Set{Int64} with 3 elements:
  2
  3
  1

In [253]:
c = Circuit()

In [254]:
a = QRegister("a", 3)
b = QRegister("c", 4)

QRegister("c", 4, Dict{Int64, HiQuER.QubitId}(4 => (0), 2 => (0), 3 => (0), 1 => (0)))

In [255]:
push!(c, a)

In [256]:
push!(c, CNOT, [("a", 1), ("a", 3)])

In [257]:
c

1: CNOT(1, 3)

In [221]:
push!(c, H[5])

In [222]:
HiQuER.qubits(c)

Set{Int64} with 4 elements:
  5
  2
  3
  1

In [223]:
push!(c, b)

1[4]
4
2Int64[]
6
3Int64[]
7
4Int64[]
8


In [224]:
c.registers

2-element Vector{QRegister}:
 QRegister("a", 3, Dict{Int64, HiQuER.QubitId}(2 => (2), 3 => (3), 1 => (1)))
 QRegister("c", 4, Dict{Int64, HiQuER.QubitId}(4 => (8), 2 => (6), 3 => (7), 1 => (4)))

In [159]:
x = HiQuER.qubits(c)

Set{Int64}()

In [161]:
if length(x) == 0
    print(2)
end

2

In [138]:
push!(a, 1, 2, 3, 4)

Set{Int64} with 4 elements:
  4
  2
  3
  1

In [121]:
Set([typeof(x) for x in qobj.prog])

Set{DataType} with 3 elements:
  OpenQASM.Types.RegDecl
  OpenQASM.Types.Include
  OpenQASM.Types.Instruction

In [78]:
parse(Float64, "0.32")

0.32

In [76]:
Rx(FloatAngle(0.3))

Rx(0.3)

In [65]:
v = 3
q = @match v begin
    1 => "blib"
    3 => "floo"
end

"floo"

In [140]:
a = QRegister("a", 3)

QRegister("a", 3, Dict{Int64, HiQuER.QubitId}(2 => (0), 3 => (0), 1 => (0)))

In [148]:
[idx for x in values(a.id_map) for idx in x]

3-element Vector{Int64}:
 0
 0
 0

In [26]:
qobj = load_qasm("./qasm/plasma4.qasm");

In [58]:
prog = qobj.prog

226-element Vector{Any}:
 include "qelib1.inc";
 qreg q[4];
 s q[0];
 h q[2];
 h q[3];
 s q[1];
 cx q[2], q[1];
 cx q[3], q[0];
 s q[0];
 s q[2];
 s q[3];
 s q[1];
 h q[0];
 ⋮
 cx q[0], q[1];
 cx q[1], q[2];
 rz(-(0.008098347207411006)) q[2];
 cx q[1], q[2];
 cx q[0], q[1];
 h q[0];
 h q[1];
 sdg q[0];
 sdg q[1];
 cx q[1], q[0];
 sdg q[0];
 h q[1];

In [126]:
HiQuER.qubits(c)

Set{Int64}()

In [128]:
c = Circuit()

In [129]:
push!(c, H[1]*T[3])

In [135]:
HiQuER.qubits(c)

Set{Int64} with 2 elements:
  3
  1

In [133]:
values(x)

ValueIterator for a Dict{Int64, Int64} with 2 entries. Values:
  3
  2

In [59]:
filter(x -> x isa OpenQASM.Types.RegDecl, prog)

1-element Vector{Any}:
 qreg q[4];

In [84]:
typeof(qobj.prog[2])

OpenQASM.Types.RegDecl

In [88]:
qobj.prog[2].size

RBNF.Token{:int}(3, 8, 44, "4", 1)

In [85]:
qobj.prog[2].name.str

"q"

In [52]:
typeof(qobj.prog[3].qargs[1])

OpenQASM.Types.Bit

In [55]:
typeof(qobj.prog[20])

OpenQASM.Types.Instruction

In [63]:
qobj.prog[3].name

"s"

In [83]:
qobj.prog[20].qargs[1].address

RBNF.Token{:int}(21, 28, 246, "3", 1)

In [89]:
a = QRegister("qa", 2)
b = QRegister("qb", 2)

QRegister("qb", 2, Dict{Int64, HiQuER.QubitId}(2 => (0), 1 => (0)))

In [90]:
c = Circuit(a, b)

In [115]:
CNOT[a[1], b[1]]

LoadError: MethodError: no method matching getindex(::ControlledGate, ::HiQuER.QubitId, ::HiQuER.QubitId)
[0mClosest candidates are:
[0m  getindex(::G, ::HiQuER.QubitId) where G<:HiQuER.AbstractGate at ~/GitHub/HiQuER/src/circuits/gate_types.jl:163
[0m  getindex(::G, [91m::Int64...[39m) where G<:HiQuER.AbstractGate at ~/GitHub/HiQuER/src/circuits/gate_types.jl:162

In [114]:
collect(flatten(x))

2-element Vector{Int64}:
 1
 3

In [110]:
[u.id for u in x]

2-element Vector{Tuple{Int64}}:
 (1,)
 (3,)

In [103]:
x = CNOT[1,2]

(1, 2) => CNOT

In [102]:
typeof(x[1])

HiQuER.QubitId

In [93]:
H[1]

(1) => H